Create a [leaflet](http://leafletjs.com/) map with the country colors according to the number of received abstracts.

**To fix:** the `geolite2.lookup()` returns      
`TypeError: a bytes-like object is required, not 'str'`


# Import modules

In [28]:
import folium
import json
import pandas as pd
from geopy.geocoders import Nominatim
from geoip import geolite2
import pycountry
import re
from collections import Counter 
folium.__version__

'0.3.0'

* [Folium](https://pypi.python.org/pypi/folium) to visualise with Leaflet,
* [json](https://docs.python.org/2/library/json.html) to read country information,
* [pandas](http://pandas.pydata.org/) to read the abstract data,
* [geopy](https://pypi.python.org/pypi/geopy) to get coordinates from location,
* [geopip](https://pypi.python.org/pypi/GeoIP/) to get the location from the IP address,
* [pycountry](https://pypi.python.org/pypi/pycountry) to change the country name into country code.

# Files I/O

## Input files

Two input files are necessary: 
* *abstractfile*, which contains a list of JSON files corresponding to the submitted abstract (can be obtained with ls *json > abstractfile);
* *geo_json_data*, a file containing the country boundaries in JSON format. Can be obtained from https://github.com/johan/world.geo.json.

In [9]:
abstractfile = '../data/abstracts/abstractslist2016.txt'
geo_json_data = r'countries.geo.json'

## Output file

* *mapfile*, a html file showing the final map.

We will create a new html file according to the considered year.

In [10]:
match = re.search(r'\d{4}', abstractfile)
year = match.group()
mapfile = 'countryabstract' + year + '.html'

# Create a list of countries from the list of abstracts

See [plot_abstracts_map.ipynb](./plot_abstracts_map.ipynb) for more details on this part.<br>
The goal of this step is to obtain the number of abstracts for each country, that will be stored in the counter object *abstract_countrycount*.

In [25]:
countrylist = []
nlines = 0
with open(abstractfile, 'rb') as f:
    for lines in f.readlines():
        print(nlines)
        nlines += 1
        match = re.search(b'(\d{4})-(\d{2})-(\d{2})_(\d{2}):(\d{2}):(\d{2})_IP_(\d+\.\d+\.\d+\.\d+).json', lines)
        if match:
            IP = match.group(7)
            print(IP)
            print(type(IP))
            matchIP = c
            if matchIP: 
                countrylist.append(matchIP.country)
abstract_countrycount = Counter(countrylist)

0
b'41.215.80.194'
<class 'bytes'>


TypeError: a bytes-like object is required, not 'str'

In [32]:
geolite2.lookup(str.encode('17.0.0.1'))

TypeError: a bytes-like object is required, not 'str'

We convert the dictionary into a CSV file (loop on the keys and values of the dictionnary; probably another way to do it).

In [ ]:
countryfile = 'countryabstract' + year + '.csv'
with open(countryfile, 'w') as f:
    f.write('Country,Abstracts\n')
    for ii, jj in zip(abstract_countrycount.keys(), abstract_countrycount.values()):
        countrycode = pycountry.countries.get(alpha2=ii).alpha3
        f.write(countrycode + ',' + str(jj) + '\n')

We read the number of abstracts by country using pandas.

In [ ]:
abstract_data = pd.read_csv(countryfile)

# Create the map

We initialize the map with the centered on the Colloquiun venue and with a global view (zoom = 1.5).

In [ ]:
geolocator = Nominatim()
location = geolocator.geocode("Place du XX Aout, Liege, Belgium")

In [ ]:
map_clq = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=1.5)
map_clq.simple_marker([location.latitude, location.longitude], popup='Colloquium Venue')
#map_clq.geo_json('countries.geo.json')

Then we add the information on the abstracts stored in a the geo JSON file (countries) and in the abstract count.

In [ ]:
map_clq.geo_json(geo_path=geo_json_data, data=abstract_data,
             columns=['Country','Abstracts'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.5, line_opacity=0.2,
             legend_name='Number of submitted abstracts',
             reset='True')
map_clq.create_map(mapfile)

In [ ]:
map_clq